In [31]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")
import os
import torch
import torchvision 
import h5py
import torchvision.transforms as transforms
import numpy as np
import keras

from dataset import get_mean_std

from keras.datasets import mnist, cifar10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0.0 Load MNIST

In [40]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [41]:
img_channels, img_rows, img_cols = 1, 28, 28 # mnist
# img_channels, img_rows, img_cols = 3, 32, 32 #cifar
X_train = X_train.reshape(X_train.shape[0], img_channels, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], img_channels, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])
X_train = np.vstack((X_train,X_test))
y_true = np.hstack((y_train,y_test))
y_true = y_true.astype('int64')

In [42]:
print(X_train.shape)
print(y_true.shape)
merge_data = X_train
merge_targets = y_true

(70000, 1, 28, 28)
(70000,)


In [35]:
merge_data = merge_data.reshape(merge_data.shape[0], -1, merge_data.shape[1], merge_data.shape[2])
print(merge_data.shape)
print(merge_targets.shape)

(70000, 28, 1, 28)
(70000,)


In [43]:
get_mean_std(merge_data, ratio=1)

(array([0.13092558], dtype=float32), array([0.3084485], dtype=float32))

## 0.1 Create MNIST dataset

In [44]:
data_dir = "../data"
dataset_name = "mnist.hdf5"
ab_path = os.path.join(data_dir, dataset_name)

In [45]:
# Image for pytorch Dataset should be (samples, height, width, channels)
merge_data = merge_data.transpose((0, 2, 3, 1))
print(merge_data.shape)

(70000, 28, 28, 1)


In [46]:
with h5py.File(ab_path, "w") as f:
    grp = f.create_group("mnist")
    grp.create_dataset("data", merge_data.shape, dtype=merge_data.dtype)
    grp.create_dataset("targets", merge_targets.shape, dtype=merge_targets.dtype)
    grp["data"][:,:,:,:] = merge_data
    grp["targets"][:] = merge_targets

## 1.0 Load Cifar10

In [23]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() #cifar10
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [25]:
img_channels, img_rows, img_cols = 3, 32, 32 #cifar
X_train = X_train.reshape(X_train.shape[0], img_channels, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], img_channels, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])
X_train = np.vstack((X_train,X_test))
y_true = np.hstack((y_train,y_test))
y_true = y_true.astype('int64')

In [26]:
print(X_train.shape)
print(y_true.shape)
merge_data = X_train
merge_targets = y_true

(60000, 3, 32, 32)
(60000,)


In [27]:
merge_data = merge_data.reshape(merge_data.shape[0], -1, merge_data.shape[1], merge_data.shape[2])
print(merge_data.shape)
print(merge_targets.shape)

(60000, 32, 3, 32)
(60000,)


# 1.1 Create Cifar10 dataset

In [28]:
data_dir = "../data"
dataset_name = "cifar10.hdf5"
ab_path = os.path.join(data_dir, dataset_name)

In [29]:
# Image for pytorch Dataset should be (samples, height, width, channels)
merge_data = merge_data.transpose((0, 2, 3, 1))
print(merge_data.shape)

(60000, 3, 32, 32)


In [30]:
with h5py.File(ab_path, "w") as f:
    grp = f.create_group("cifar10")
    grp.create_dataset("data", merge_data.shape, dtype=merge_data.dtype)
    grp.create_dataset("targets", merge_targets.shape, dtype=merge_targets.dtype)
    grp["data"][:,:,:,:] = merge_data
    grp["targets"][:] = merge_targets